In [ ]:
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta
import ipywidgets as widgets

%matplotlib widget
pd.options.display.max_columns = None

---

# 06 Heikin-Ashi Bull Pin 

Click $>>$ button to run the entire script first

**Buy Signal**
> BUY 1 share when open equal to Low and close lower than High and high higher than Prev N(high,-2,day) at hour candle interval using Heikin-Ashi chart. Enter trade between 09:00 to 23:59

**Sell Signal**
> SELL 1 share at stop loss % of 1.0 or target profit % of 2.0 at hour candle interval using Heikin-Ashi chart.

Please feel free to change to any **NSE SYMBOL** to the below list, separated by a comma

In [ ]:
symbols = "ITC"

In [ ]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "14d"
intervalOfData = "1h"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas

## A Heikin Ashin Chart

<img src="https://static.incrediblecharts.com/images/png_images/swks-heikin-ashi.png" width="900" height="300"/>

### Buy signal
> BUY 1 share when open equal to Low and close lower than High and high higher than Prev N(high,-2,day) at hour candle interval using Heikin-Ashi chart. Enter trade between 09:00 to 23:59

Here $HA$ stands for Heikin Ashin chart open,high,low,close at the hourly time chart unless otherwise specified

- if $HA.open == HA.low$
    - if $HA.close < HA.high$
        - if $HA.high > HA.high[-2] |_{day}$
    
### Sell signal
> SELL 1 share at stop loss % of 1.0 or target profit % of 2.0 at hour candle interval using Heikin-Ashi chart.

If either of the below conditions are met
- Stop Loss condition met if $HA.close[0] < order.buy*0.99$
- Target price condition met if $HA.close[0] > order.buy*1.02$

In [ ]:
display(Markdown("# Set the variables to this strategy here"))
display(Markdown("> We will improve UI shortly"))
display(Markdown("If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly"))
@widgets.interact(stopLoss=widgets.FloatSlider(
                                    value=1,
                                    min=0,
                                    max=3,
                                    step=0.01,
                                    description='Stop Loss %:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='vertical',
                                    readout=True,
                                    readout_format='.2f',
                                ),\
                  targetPrice=widgets.FloatSlider(
                                    value=2,
                                    min=0,
                                    max=3,
                                    step=0.01,
                                    description='Target Price %:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='vertical',
                                    readout=True,
                                    readout_format='.2f',
                                ),\
                 )
def voilas(stopLoss,targetPrice):
    class HeikinAshinBullPin(bt.Strategy):
        def __init__(self):
            # Keep a reference to the "close" line in the data[0] dataseries
            self.hour = self.datas[0]
            self.date = self.datas[1]
            #Some indicators
            self.heikinDate = bt.indicators.HeikinAshi(self.datas[0])
            self.heikinHour = bt.indicators.HeikinAshi(self.datas[1])
            # self.rsi = bt.indicators.RSI_SMA(period=3)

            # To keep track of pending orders and buy price/commission
            self.order = None
            self.buyprice = None
            self.buycomm = None

        def log(self, txt, dt=None):
            ''' Logging function for this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s | %s' % (dt.isoformat(),self.datas[0].datetime.time(0), txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                else:  # Sell
                    self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            self.order = None

        def notify_trade(self, trade):
            if not trade.isclosed:
                return

            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

        def next(self):
            # check chart 2 days ago's
            # self.log(f'{self.position}')
            if self.heikinHour.ha_open[0] == self.heikinHour.ha_low[0]:
                if self.heikinHour.ha_close[0] < self.heikinHour.ha_high[0]:
                    if self.heikinHour.ha_high[0] > self.heikinDate.ha_high[-2]:
                        self.order = self.buy(size=1)
            if self.position:
                if self.heikinHour.ha_close[0] < (1-stopLoss/100)*self.position.price_orig:
                    self.log("stoploss")
                    self.order = self.sell(size=1)
                if self.heikinHour.ha_close[0] > (1+targetPrice/100)*self.position.price_orig:
                    self.log("targetPrice")
                    self.order = self.sell(size=1)
    cerebro = bt.Cerebro()

    data = bt.feeds.PandasData(dataname=stockPandasTot[symbol+".NS"][::-1],timeframe=bt.TimeFrame.Minutes)
    cerebro.adddata(data)
    cerebro.resampledata(data, timeframe=bt.TimeFrame.Days,compression=1)
    cerebro.addstrategy(HeikinAshinBullPin)

    cerebro.broker.setcash(10000)

    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot(style="candlestick",width=20,height=1)

---
### Understanding above plot

> Please increase the size of the plot to the entire width of the screen, before reading this section. You can do the same by dragging the triangular element shown at the bottom right corner of the plot

The plot above has quite a lines and legends on them.
The legend is available on the left side indicating various items.

We shall move from the bottom most element and move upwards.
#### Volume
The volume is indicated by bars of either gray or red colour. The respective value is availabe on the left part of the plot along the y-axis. Exact values for volume cannot be extracted currently. A red bar indicates a decrease in stock price over the time period. A grey bar indicates an increase in stock price over the given time period.
#### Heikin Ashin (Hourly)
This chart is created using the famous japanese canlde stick Heikin Ashin. You can find all details pertaining to this at [this wiki](https://en.wikipedia.org/wiki/Heikin-Ashi_chart).
#### Heikin Ashin (Daily)
This chart involves the resampled hourly data to that of Daily.
#### Triangles
The triangles represent positions taken up according to the strategy mentioned by code above. A buy position/ signal will be generated if the triangle is green and pointed upwards, while a sell position/ sell signal is generated with the red downward pointing triangle.
#### Trading dots
The next canvas involves a set of dots representing either profit or losses which have occured by following the strategy. Here too the magnitudes can be separated by the colour of the dot and the cursor value on hovering over dots. One can also approximate the values with the data on the scale located on the right.
#### Portfolio value
The final screen on the top indicates the portfolio value and cash value. Each are differentiated by the respective color.